In [ ]:
!git clone https://github.com/martenjostmann/da2-case-study-2.git
!cd "da2-case-study-2"
!mkdir data
!wget "https://uni-muenster.sciebo.de/s/KNjoOrXdWecbvdh/download?path=%2F&files=validation_data.zip" -O data/temp.zip
!unzip data/temp.zip -d data
!rm data/temp.zip

fatal: destination path 'da2-case-study-2' already exists and is not an empty directory.
mkdir: cannot create directory ‘data’: File exists
--2022-07-03 13:15:11--  https://uni-muenster.sciebo.de/s/KNjoOrXdWecbvdh/download?path=%2F&files=validation_data.zip
Resolving uni-muenster.sciebo.de (uni-muenster.sciebo.de)... 128.176.4.4
Connecting to uni-muenster.sciebo.de (uni-muenster.sciebo.de)|128.176.4.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 334076991 (319M) [application/zip]
Saving to: ‘data/temp.zip’

data/temp.zip       100%[===================>] 318.60M  26.3MB/s    in 13s     

2022-07-03 13:15:24 (24.7 MB/s) - ‘data/temp.zip’ saved [334076991/334076991]

Archive:  data/temp.zip
   creating: data/02_validation_data_images/
  inflating: data/02_validation_data_images/UDPYYD.png  
  inflating: data/02_validation_data_images/DQIMQN.png  
  inflating: data/02_validation_data_images/DQIMQN.csv  
  inflating: data/02_validation_data_images/UDPYYD.csv  
 

In [ ]:
data_path = "data/02_validation_data_images"
image_file_format = "png"

In [ ]:
from PIL import Image
import glob
import os
import numpy as np

from tensorflow import keras

from src.d01_data import save_data
from src.d02_processing import sliding_window, postprocess
from src.d05_evaluation import evaluate_predictions

In [ ]:
# Load images

images = []
for filename in glob.glob(data_path + '/*.' + image_file_format):
    image=Image.open(filename)
    images.append((np.asarray(image), filename))

In [ ]:
# Load model
model = keras.models.load_model('models','efficientnet_v2_best')

In [ ]:
# Make predictions
for image, file_path in images:
    # Get file name
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    output_path = f"data/{file_name}_pred.csv"

    print(f"Predicting patches for {file_name}...")
    pred, confidence = sliding_window.find_classes(image, model, 256, 121, (224, 224))
    boxes = postprocess.merge_predictions(pred, confidence, min_confidence=0.6, n_range_per_class={1:1, 2:1, 3:1, 4:2})

    save_data.save_boxes(pred, 121, output_path)

In [ ]:
# Printing the F1-Scores
for image, file_path in images:
    # Get file name
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    gt_path = os.path.join(os.getcwd(), 'data', file_name + '.csv')
    pred_path = os.path.join(os.getcwd(), 'data', file_name + '_pred.csv')
    score = evaluate_predictions.calc_performance(gt_path, pred_path)
    print(file_name + ":", score)